In [13]:
import pandas as pd
import numpy as np
import joblib
import mlflow
import mlflow.sklearn

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from xgboost import XGBClassifier

# Load dataset
df = pd.read_csv("hand_landmarks_data.csv")

# Map labels
def map_label(label):
    if label == "like":
        return "up"
    elif label == "dislike":
        return "down"
    elif label == "peace":
        return "right"
    elif label == "stop":
        return "left"
    else:
        return None  # will drop these later


df['label'] = df['label'].apply(map_label)
df = df.dropna(subset=['label'])


# Center by wrist (x1, y1, z1)
wrist_x, wrist_y, wrist_z = df['x1'], df['y1'], df['z1']
for i in range(1, 22):
    df[f'x{i}'] -= wrist_x
    df[f'y{i}'] -= wrist_y
    df[f'z{i}'] -= wrist_z

# Normalize by max absolute value
max_value = df.iloc[:, :-1].abs().max(axis=1)
for i in range(1, 22):
    df[f'x{i}'] /= max_value
    df[f'y{i}'] /= max_value
    df[f'z{i}'] /= max_value

# Prepare data
X = df.drop(columns=['label'])
y = df['label']

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Save label encoder
joblib.dump(label_encoder, "label_encoder.pkl")

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train XGBoost model
model = XGBClassifier(max_depth=5, learning_rate=0.1, n_estimators=200, random_state=42)
model.fit(X_train, y_train)

# Evaluate
preds = model.predict(X_test)
acc = accuracy_score(y_test, preds)
f1 = f1_score(y_test, preds, average="macro")

print(f"Accuracy: {acc:.4f}, F1-score: {f1:.4f}")

# Save model
joblib.dump(model, "gesture_model.pkl")


Accuracy: 0.9982, F1-score: 0.9982


['gesture_model.pkl']

In [14]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import joblib
import numpy as np

# Load model and label encoder
model = joblib.load("gesture_model.pkl")
label_encoder = joblib.load("label_encoder.pkl")

app = FastAPI()

class Landmarks(BaseModel):
    landmarks: list[float]

@app.post("/predict")
def predict(data: Landmarks):
    if len(data.landmarks) != 63:
        raise HTTPException(status_code=400, detail="Invalid number of landmarks, expected 63 floats")

    X_input = np.array(data.landmarks).reshape(1, -1)
    pred_index = model.predict(X_input)[0]
    gesture = label_encoder.inverse_transform([pred_index])[0]

    return {"prediction": gesture}


In [15]:
landmarks = [round(i * 0.01, 2) for i in range(1, 64)]
print(landmarks)


[0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3, 0.31, 0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4, 0.41, 0.42, 0.43, 0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5, 0.51, 0.52, 0.53, 0.54, 0.55, 0.56, 0.57, 0.58, 0.59, 0.6, 0.61, 0.62, 0.63]
